In [4]:
import pandas as pd
from rdkit import Chem, rdBase
from rdkit.Chem import rdFMCS
from rdkit.Chem import rdRGroupDecomposition as rdRGD

from rgroupinterm.rgroupenumeration import EnumRGroups

rdBase.DisableLog('rdApp.*')

In [5]:
smiles = [
    'CC(C)(C)c1ccc2c(c1)[C@H]1OCCC[C@H]1[C@H](c1ccc(O)cc1)N2',
    '[NH3+]C[C@H]1CC[C@@H]2[C@H](O1)c1cc(C(F)(F)F)ccc1N[C@H]2c1ccccc1'
]
mols = [Chem.MolFromSmiles(x) for x in smiles]

generator = EnumRGroups()

df_interm, core = generator.generate_intermediates(mols)
intermediates = df_interm['Intermediate'].tolist()

{2: rdkit.Chem.rdchem.ChiralType.CHI_UNSPECIFIED, 1: rdkit.Chem.rdchem.ChiralType.CHI_UNSPECIFIED, 3: rdkit.Chem.rdchem.ChiralType.CHI_UNSPECIFIED}
{2: rdkit.Chem.rdchem.ChiralType.CHI_UNSPECIFIED, 1: rdkit.Chem.rdchem.ChiralType.CHI_UNSPECIFIED, 3: rdkit.Chem.rdchem.ChiralType.CHI_UNSPECIFIED}
{2: rdkit.Chem.rdchem.ChiralType.CHI_TETRAHEDRAL_CW, 1: rdkit.Chem.rdchem.ChiralType.CHI_UNSPECIFIED, 3: rdkit.Chem.rdchem.ChiralType.CHI_UNSPECIFIED}
{2: rdkit.Chem.rdchem.ChiralType.CHI_TETRAHEDRAL_CW, 1: rdkit.Chem.rdchem.ChiralType.CHI_UNSPECIFIED, 3: rdkit.Chem.rdchem.ChiralType.CHI_UNSPECIFIED}
{2: rdkit.Chem.rdchem.ChiralType.CHI_UNSPECIFIED, 1: rdkit.Chem.rdchem.ChiralType.CHI_UNSPECIFIED, 3: rdkit.Chem.rdchem.ChiralType.CHI_UNSPECIFIED}
{2: rdkit.Chem.rdchem.ChiralType.CHI_UNSPECIFIED, 1: rdkit.Chem.rdchem.ChiralType.CHI_UNSPECIFIED, 3: rdkit.Chem.rdchem.ChiralType.CHI_UNSPECIFIED}
{2: rdkit.Chem.rdchem.ChiralType.CHI_TETRAHEDRAL_CW, 1: rdkit.Chem.rdchem.ChiralType.CHI_UNSPECIFIED, 3: r

In [6]:
len(intermediates)

8